In [1]:
import gc
import os
import pathlib
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
from tqdm import tqdm
from typing import Dict, List, Tuple, NamedTuple, Callable, Any
import scml
from scml import pandasx as pdx
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()

In [2]:
sentences = [
    "Today the majority of humans own and operate cell phones on a daily basis. In essay form, explain if drivers should or should not be able to use cell phones in any capacity while operating a vehicle.",
    "Write an explanatory essay to inform fellow citizens about the advantages of limiting car usage. Your essay must be based on ideas and information that can be found in the passage set. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your essay in the space provided.",
    "Some schools require students to complete summer projects to assure they continue learning during their break. Should these summer projects be teacher-designed or student-designed? Take a position on this question. Support your response with reasons and specific examples.",
    "You have just read the article, 'A Cowboy Who Rode the Waves.' Luke's participation in the Seagoing Cowboys program allowed him to experience adventures and visit many unique places. Using information from the article, write an argument from Luke's point of view convincing others to participate in the Seagoing Cowboys program. Be sure to include: reasons to join the program; details from the article to support Luke's claims; an introduction, a body, and a conclusion to your essay.",
    "Your principal has decided that all students must participate in at least one extracurricular activity. For example, students could participate in sports, work on the yearbook, or serve on the student council. Do you agree or disagree with this decision? Use specific details and examples to convince others to support your position.",
    "In 'The Challenge of Exploring Venus,' the author suggests studying Venus is a worthy pursuit despite the dangers it presents. Using details from the article, write an essay evaluating how well the author supports this idea. Be sure to include: a claim that evaluates how well the author supports the idea that studying Venus is a worthy pursuit despite the dangers; an explanation of the evidence from the article that supports your claim; an introduction, a body, and a conclusion to your essay.",
    "In the article 'Making Mona Lisa Smile,' the author describes how a new technology called the Facial Action Coding System enables computers to identify human emotions. Using details from the article, write an essay arguing whether the use of this technology to read the emotional expressions of students in a classroom is valuable.",
    "You have read the article 'Unmasking the Face on Mars.' Imagine you are a scientist at NASA discussing the Face with someone who thinks it was created by aliens. Using information in the article, write an argumentative essay to convince someone that the Face is just a natural landform.Be sure to include: claims to support your argument that the Face is a natural landform; evidence from the article to support your claims; an introduction, a body, and a conclusion to your argumentative essay.",
    "Some of your friends perform community service. For example, some tutor elementary school children and others clean up litter. They think helping the community is very important. But other friends of yours think community service takes too much time away from what they need or want to do. Your principal is deciding whether to require all students to perform community service. Write a letter to your principal in which you take a position on whether students should be required to perform community service. Support your position with examples.",
    "Your principal is considering changing school policy so that students may not participate in sports or other activities unless they have at least a grade B average. Many students have a grade C average. She would like to hear the students' views on this possible policy change. Write a letter to your principal arguing for or against requiring at least a grade B average to participate in sports or other activities. Be sure to support your arguments with specific reasons.",
    "In the article 'Driverless Cars are Coming,' the author presents both positive and negative aspects of driverless cars. Using details from the article, create an argument for or against the development of these cars. Be sure to include: your position on driverless cars; appropriate details from the article that support your position; an introduction, a body, and a conclusion to your argumentative essay.",
    "Write a letter to your state senator in which you argue in favor of keeping the Electoral College or changing to election by popular vote for the president of the United States. Use the information from the texts in your essay. Manage your time carefully so that you can read the passages; plan your response; write your response; and revise and edit your response. Be sure to include a claim; address counterclaims; use evidence from multiple sources; and avoid overly relying on one source. Your response should be in the form of a multiparagraph essay. Write your response in the space provided.",
    "Your principal is reconsidering the school's cell phone policy. She is considering two possible policies: Policy 1: Allow students to bring phones to school and use them during lunch periods and other free times, as long as the phones are turned off during class time. Policy 2: Do not allow students to have phones at school at all. Write a letter to your principal convincing her which policy you believe is better. Support your position with specific reasons.",
    "Some schools offer distance learning as an option for students to attend classes from home by way of online or video conferencing. Do you think students would benefit from being able to attend classes from home? Take a position on this issue. Support your response with reasons and examples.",
    "When people ask for advice, they sometimes talk to more than one person. Explain why seeking multiple opinions can help someone make a better choice. Use specific details and examples in your response.",
]

In [3]:
df = pd.read_parquet("input/preprocess.parquet")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43529 entries, 0 to 43528
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   essay_id        43529 non-null  int32 
 1   generated       43529 non-null  int8  
 2   source          43529 non-null  object
 3   prompt          43529 non-null  object
 4   text            43529 non-null  object
 5   text_bsc        43529 non-null  object
 6   text_bow        43529 non-null  object
 7   text_bow_len    43529 non-null  int16 
 8   prompt_bsc      43529 non-null  object
 9   prompt_bow      43529 non-null  object
 10  prompt_bow_len  43529 non-null  int16 
dtypes: int16(2), int32(1), int8(1), object(7)
memory usage: 2.7+ MB


In [4]:
pdx.value_counts(df["source"])

,count,percent
source,,
persuade_corpus,25793,0.592547
mistral7binstruct_v1,2420,0.055595
llama2_chat,2418,0.055549
mistral7binstruct_v2,2409,0.055342
original_moth,2264,0.052011
chat_gpt_moth,2124,0.048795
train_essays,1378,0.031657
llama_70b_v1,1172,0.026925
falcon_180b_v1,1051,0.024145


In [5]:
white = ["persuade_corpus", "train_essays"]
min_len = 1394  # P20
tmp = df[(df["generated"]==0) & (df["source"].isin(white)) & (df["text_bow_len"]>=min_len)]
total = len(tmp)
#tmp = tmp.sample(16_000)
sentences += tmp["text_bow"].tolist()
print(f"{len(sentences):,} sentences")

Sampled 21,222 out of 21,207 sentences


# Build Index

In [6]:
%%time
model = SentenceTransformer("huggingface/sentence-transformers/all-mpnet-base-v2", device=torch.device("mps"))
model.max_seq_length = 384
em = model.encode(sentences=sentences, batch_size=512, show_progress_bar=True, convert_to_numpy=True)
print(f"em={em.shape}")

Batches:   0%|          | 0/42 [00:00<?, ?it/s]

em=(21222, 768)
CPU times: user 17.7 s, sys: 4.13 s, total: 21.8 s
Wall time: 6min 3s


In [7]:
faiss.normalize_L2(em)

In [8]:
%%time
d = em.shape[1]
index = faiss.IndexFlatIP(d)  # InnerProduct
index.verbose = True
index.train(em)
index.add(em)

CPU times: user 3.58 ms, sys: 7.84 ms, total: 11.4 ms
Wall time: 11.3 ms


In [9]:
%%time
faiss.write_index(index, "output/white.index")

CPU times: user 235 µs, sys: 8.22 ms, total: 8.45 ms
Wall time: 10.6 ms


# Sanity Check

In [10]:
%%time
index = faiss.read_index("output/white.index")
print(f"ntotal={index.ntotal}, is_trained={index.is_trained}")

ntotal=21222, is_trained=True
CPU times: user 1.26 ms, sys: 11.9 ms, total: 13.2 ms
Wall time: 18.7 ms


In [11]:
k=1
distances, ids = index.search(em[:20], k)  
print(f"I={repr(ids)}\nD={repr(distances)}")

I=array([[ 0],
       [ 1],
       [ 2],
       [ 3],
       [ 4],
       [ 5],
       [ 6],
       [ 7],
       [ 8],
       [ 9],
       [10],
       [11],
       [12],
       [13],
       [14],
       [15],
       [16],
       [17],
       [18],
       [19]])
D=array([[1.0000002 ],
       [0.99999994],
       [1.0000006 ],
       [0.99999994],
       [1.0000002 ],
       [1.0000001 ],
       [1.0000004 ],
       [1.0000005 ],
       [1.0000002 ],
       [1.0000001 ],
       [1.        ],
       [1.0000007 ],
       [1.        ],
       [1.        ],
       [1.0000002 ],
       [1.0000005 ],
       [1.0000004 ],
       [0.99999994],
       [1.0000004 ],
       [1.0000007 ]], dtype=float32)


In [12]:
queries = ["phone", "mars", "venus", "school projects", "school sports", "school activities", "cowboy", "facial emotion", "planets", "community service",
          "cars", "automobile", "tesla", "school policy", "politician", "government", "activist", "asking for advice", "distance learning", "learning outside the classroom",
           "baking class", 
          ]
q = model.encode(sentences=queries, batch_size=512, show_progress_bar=True, convert_to_numpy=True)
print(f"q={q.shape}")
faiss.normalize_L2(q)
k=1
distances, ids = index.search(q, k)
distances = distances.flatten()
ids = ids.flatten()
scores = list(zip(distances, queries))
scores.sort()
print(scores)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

q=(21, 768)
[(0.31021312, 'baking class'), (0.32608566, 'politician'), (0.3469167, 'activist'), (0.3492005, 'government'), (0.37236774, 'school projects'), (0.46479127, 'cowboy'), (0.48198825, 'cars'), (0.50803983, 'automobile'), (0.5106132, 'planets'), (0.51134557, 'tesla'), (0.52504694, 'mars'), (0.54078025, 'phone'), (0.55978537, 'community service'), (0.5989828, 'school sports'), (0.61108714, 'learning outside the classroom'), (0.6640553, 'distance learning'), (0.6758558, 'school policy'), (0.6805871, 'venus'), (0.6867367, 'school activities'), (0.69437516, 'facial emotion'), (0.7366042, 'asking for advice')]


# Train/Validation Split

In [13]:
%%time
em = model.encode(sentences=df["text_bow"].tolist(), batch_size=512, show_progress_bar=True, convert_to_numpy=True)
faiss.normalize_L2(em)
print(f"em={em.shape}")

Batches:   0%|          | 0/86 [00:00<?, ?it/s]

em=(43529, 768)
CPU times: user 32 s, sys: 11.6 s, total: 43.6 s
Wall time: 12min 41s


In [14]:
%%time
k=1
distances, ids = index.search(em, k)  
col = "white_sim"
df[col] = distances.flatten().tolist()
df[col] = df[col].astype(np.float32)
df[col].describe(percentiles=percentiles)

CPU times: user 1.3 s, sys: 77.2 ms, total: 1.38 s
Wall time: 556 ms


count    43529.000000
mean         0.865916
std          0.178425
min          0.199636
1%           0.416369
5%           0.493065
10%          0.550357
20%          0.674016
30%          0.850393
40%          0.905675
50%          0.946312
60%          1.000000
70%          1.000000
80%          1.000000
90%          1.000000
95%          1.000000
99%          1.000001
max          1.000001
Name: white_sim, dtype: float64

# Review Data

In [15]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43529 entries, 0 to 43528
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   essay_id        43529 non-null  int32  
 1   generated       43529 non-null  int8   
 2   source          43529 non-null  object 
 3   prompt          43529 non-null  object 
 4   text            43529 non-null  object 
 5   text_bsc        43529 non-null  object 
 6   text_bow        43529 non-null  object 
 7   text_bow_len    43529 non-null  int16  
 8   prompt_bsc      43529 non-null  object 
 9   prompt_bow      43529 non-null  object 
 10  prompt_bow_len  43529 non-null  int16  
 11  white_sim       43529 non-null  float32
dtypes: float32(1), int16(2), int32(1), int8(1), object(7)
memory usage: 2.9+ MB


In [16]:
df.groupby(["source"])["white_sim"].agg(["min", "median", "max"])

,min,median,max
source,,,
chat_gpt_moth,0.236391,0.602091,0.956580
darragh_claude_v6,0.753740,0.913299,0.974308
darragh_claude_v7,0.740141,0.915620,0.972452
falcon_180b_v1,0.199636,0.656130,0.968698
llama2_chat,0.278713,0.626164,0.928801
llama_70b_v1,0.357002,0.665119,0.968314
mistral7binstruct_v1,0.226915,0.626436,0.937744
mistral7binstruct_v2,0.242582,0.615960,1.000000
original_moth,0.301098,0.608155,0.996677


In [17]:
df.groupby(["generated"])["white_sim"].agg(["min", "median", "max"])

,min,median,max
generated,,,
0,0.301098,1.000000,1.000001
1,0.199636,0.668244,1.000000


In [18]:
%%time
df.to_parquet(f"output/white.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 613 ms, sys: 93.2 ms, total: 706 ms
Wall time: 759 ms


In [19]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 0:18:47.366334
